In [1]:
from tensorflow import keras
import gc

D:\ANACONDA\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\ANACONDA\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
D:\ANACONDA\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
from CustomLayers import *
from GS_composer import *
from Functions import *
from ClassModel import *

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [3]:
user_profile = "E:/learning resource/"
Model_path = user_profile+"OneDrive - The University of Queensland/PhD/HPC_Results/Sugarcane_disease/ML/1Head_RealForm32_32LocalB"
#Model_path = "E:/learning resource/OneDrive - The University of Queensland/PhD/HPC_Results/Sugarcane_disease/ML/"

In [4]:
from keras import backend as K

In [14]:
model_folder = "pachy_MultiHeadAttentionLNN_1"
#model_folder = "test_model"
full_path = Model_path + "/" + model_folder
model = keras.models.load_model(full_path,custom_objects={"MultiHead_QKV_BlockAttention": MultiHead_QKV_BlockAttention,
                                                         "SNPBlockLayer":SNPBlockLayer})

[TensorShape([None, 2608, 32])]
[TensorShape([None, 2608, 32]), TensorShape([None, 2608, 32])]


In [ ]:
model.summary()
#model = keras.Model(inputs=input1, outputs=QV_output)
model.compile(optimizer="RMSprop", loss="mean_squared_error")

In [17]:

data = np.ones((1, 26086, 1))
model.predict(data)

C:\Users\pc\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\data\ops\dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


array([[3.7426164]], dtype=float32)

In [ ]:
intermediate_layer_model = tf.keras.Model(inputs=model.input,
                                         outputs=model.get_layer('add_3').output)

In [ ]:
intermediate_output = intermediate_layer_model.predict(data)

In [ ]:
attention = intermediate_output[0]
attention = K.expand_dims(attention,axis=1)
print(attention.shape)

In [ ]:
LNN_weight = model.layers[2].get_weights()

In [ ]:
LNN_weight[0].shape

In [ ]:
SNP_attention = tf.multiply(attention,LNN_weight[0])

In [ ]:
SNP_attention = tf.reshape(SNP_attention,(26080,1,32))

In [ ]:
mean_SNP_attention = SNP_attention #tf.reduce_mean(SNP_attention,axis=2)
mean_SNP_attention.shape
mean_SNP_attention = np.array(tf.reshape(mean_SNP_attention,(26080*32,)))

In [ ]:
import matplotlib.pyplot as plt
import math

In [ ]:
print(mean_SNP_attention.shape)
plt.scatter(x=np.tile(np.arange(0,26080),32),y=abs((mean_SNP_attention)))

In [ ]:
mean_SNP_attention.shape

In [ ]:
def export_attention_score_by_trait(filename,trait,val):
    model = keras.models.load_model(filename,custom_objects={"MultiHead_QKV_BlockAttention": MultiHead_QKV_BlockAttention,
                                                         "SNPBlockLayer":SNPBlockLayer})
    model.summary()
    #model = keras.Model(inputs=input1, outputs=QV_output)
    model.compile(optimizer="RMSprop", loss="mean_squared_error")
    data = np.ones((1, 26086, 1))
    intermediate_layer_model = tf.keras.Model(inputs=model.input,
                                         outputs=model.get_layer('add_3').output)
    intermediate_output = intermediate_layer_model.predict(data)
    attention = intermediate_output[0]
    attention = K.expand_dims(attention,axis=1)
    print(attention.shape)
    
    LNN_weight = model.layers[2].get_weights()
    SNP_attention = tf.multiply(attention,LNN_weight[0])
    SNP_attention = np.array(tf.reshape(SNP_attention,(32,26080)))
    SNP_attention = pd.DataFrame(SNP_attention,columns=list(range(1,SNP_attention.shape[1]+1)))
    SNP_attention.shape
    SNP_attention.insert(0,'Channel',range(1,33))
    SNP_attention.insert(0,'Val',val)
    SNP_attention.insert(0,"Trait",trait)
    
    SNP_attention = SNP_attention.values.tolist()
    #mean_SNP_attention = [trait,val]+list(np.array(tf.reshape(mean_SNP_attention,(26080,))))
    #mean_SNP_attention = pd.DataFrame([trait,val]+list(mean_SNP_attention),columns=[trait,val]+list(range(1,length(mean_SNP_attention))))

    return SNP_attention
    
    

In [22]:
def export_LD_attention_score_by_trait(filename,trait,val):
    model = keras.models.load_model(filename,custom_objects={"MultiHead_QKV_BlockAttention": MultiHead_QKV_BlockAttention,
                                                         "SNPBlockLayer":SNPBlockLayer})
    model.summary()
    #model = keras.Model(inputs=input1, outputs=QV_output)
    model.compile(optimizer="RMSprop", loss="mean_squared_error")
    data = np.ones((1, 26086, 1))
    intermediate_layer_model = tf.keras.Model(inputs=model.input,
                                         outputs=model.get_layer('multi_head_qkv__block_attention').output)
    intermediate_output = intermediate_layer_model.predict(data)
    attention = intermediate_output[0]
    attention = K.expand_dims(attention,axis=1)
    print(attention.shape)
    
    #LNN_weight = model.layers[2].get_weights()
    #SNP_attention = tf.multiply(attention,LNN_weight[0])
    LD_attention = np.array(tf.reshape(attention,(32,2608)))
    LD_attention = pd.DataFrame(LD_attention,columns=list(range(1,LD_attention.shape[1]+1)))

    LD_attention.insert(0,'Channel',range(1,33))
    LD_attention.insert(0,'Val',val)
    LD_attention.insert(0,"Trait",trait)
    
    LD_attention = LD_attention.values.tolist()
    #mean_SNP_attention = [trait,val]+list(np.array(tf.reshape(mean_SNP_attention,(26080,))))
    #mean_SNP_attention = pd.DataFrame([trait,val]+list(mean_SNP_attention),columns=[trait,val]+list(range(1,length(mean_SNP_attention))))
    model = None
    keras.backend.clear_session()
    gc.collect()
    
    return LD_attention

In [23]:
Model_path = user_profile+"OneDrive - The University of Queensland/PhD/HPC_Results/Sugarcane_disease/ML/1Head_RealForm32_32LocalB/"
records = []
for trait in ["smut","pachy"]:
    for val in range(1,6):
        filename = Model_path + trait + "_MultiHeadAttentionLNN_"+str(val)
        record = export_LD_attention_score_by_trait(filename,trait,val)
        records.append(record)


[TensorShape([None, 2608, 32])]
[TensorShape([None, 2608, 32]), TensorShape([None, 2608, 32])]
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer_1 (InputLayer)      [(None, 26086, 1)]   0                                            
__________________________________________________________________________________________________
zero_padding1d (ZeroPadding1D)  (None, 26086, 1)     0           input_layer_1[0][0]              
__________________________________________________________________________________________________
locally_connected1d (LocallyCon (None, 2608, 32)     834560      zero_padding1d[0][0]             
__________________________________________________________________________________________________
dense (Dense)                   (None, 2608, 32)     1056        locally_connected1d[0][0]        

C:\Users\pc\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\data\ops\dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


(2608, 1, 32)
[TensorShape([None, 2608, 32])]
[TensorShape([None, 2608, 32]), TensorShape([None, 2608, 32])]
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer_1 (InputLayer)      [(None, 26086, 1)]   0                                            
__________________________________________________________________________________________________
zero_padding1d (ZeroPadding1D)  (None, 26086, 1)     0           input_layer_1[0][0]              
__________________________________________________________________________________________________
locally_connected1d (LocallyCon (None, 2608, 32)     834560      zero_padding1d[0][0]             
__________________________________________________________________________________________________
dense (Dense)                   (None, 2608, 32)     1056        locally_connected1d

(2608, 1, 32)
[TensorShape([None, 2608, 32])]
[TensorShape([None, 2608, 32]), TensorShape([None, 2608, 32])]
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer_1 (InputLayer)      [(None, 26086, 1)]   0                                            
__________________________________________________________________________________________________
zero_padding1d (ZeroPadding1D)  (None, 26086, 1)     0           input_layer_1[0][0]              
__________________________________________________________________________________________________
locally_connected1d (LocallyCon (None, 2608, 32)     834560      zero_padding1d[0][0]             
__________________________________________________________________________________________________
dense (Dense)                   (None, 2608, 32)     1056        locally_connected1d

(2608, 1, 32)
[TensorShape([None, 2608, 32])]
[TensorShape([None, 2608, 32]), TensorShape([None, 2608, 32])]
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer_1 (InputLayer)      [(None, 26086, 1)]   0                                            
__________________________________________________________________________________________________
zero_padding1d (ZeroPadding1D)  (None, 26086, 1)     0           input_layer_1[0][0]              
__________________________________________________________________________________________________
locally_connected1d (LocallyCon (None, 2608, 32)     834560      zero_padding1d[0][0]             
__________________________________________________________________________________________________
dense (Dense)                   (None, 2608, 32)     1056        locally_connected1d

(2608, 1, 32)
[TensorShape([None, 2608, 32])]
[TensorShape([None, 2608, 32]), TensorShape([None, 2608, 32])]
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer_1 (InputLayer)      [(None, 26086, 1)]   0                                            
__________________________________________________________________________________________________
zero_padding1d (ZeroPadding1D)  (None, 26086, 1)     0           input_layer_1[0][0]              
__________________________________________________________________________________________________
locally_connected1d (LocallyCon (None, 2608, 32)     834560      zero_padding1d[0][0]             
__________________________________________________________________________________________________
dense (Dense)                   (None, 2608, 32)     1056        locally_connected1d

(2608, 1, 32)
[TensorShape([None, 2608, 32])]
[TensorShape([None, 2608, 32]), TensorShape([None, 2608, 32])]
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer_1 (InputLayer)      [(None, 26086, 1)]   0                                            
__________________________________________________________________________________________________
zero_padding1d (ZeroPadding1D)  (None, 26086, 1)     0           input_layer_1[0][0]              
__________________________________________________________________________________________________
locally_connected1d (LocallyCon (None, 2608, 32)     834560      zero_padding1d[0][0]             
__________________________________________________________________________________________________
dense (Dense)                   (None, 2608, 32)     1056        locally_connected1d

In [24]:
recordss = []
for i in records:
    for j in i:
        recordss.append(j)
records_df = pd.DataFrame(recordss,columns=["Trait","Val","Channel"]+list(range(1,2609)))
output_path = Model_path + "Attention_block_Normscore_bychannel.csv"
records_df.to_csv(output_path,sep="\t",index=False)

In [ ]:
recordss[0]

In [ ]:
records.insert(0,"Val", #columns.values[0:2] = ["Trait","Val"]
output_path = Model_path + "Attention_score.csv"
records.to_csv(output_path,sep="\t",index=False)

In [12]:
records_df.head(4)

,Trait,Val,Channel,1,2,3,4,5,6,7,...,2599,2600,2601,2602,2603,2604,2605,2606,2607,2608
0,smut,1,1,-1.053384,-0.956152,2.189313,-0.647232,0.594774,-1.005392,-0.623748,...,-0.573900,-0.649464,2.171458,-0.063063,-0.376861,-0.318065,1.236210,1.875976,-0.218107,0.179610
1,smut,1,2,-0.263064,-0.415714,-1.451103,-0.648146,-0.479885,0.312893,1.548072,...,1.669782,-0.979295,-0.191333,-0.702827,2.470894,-0.337153,-0.664402,-0.361315,-0.489121,-0.256168
2,smut,1,3,-1.012067,-0.915943,1.618387,-0.610754,0.235670,-0.964619,-0.587350,...,-0.602530,-0.678476,1.977512,-0.087632,0.049790,-0.608356,1.182622,1.494565,-0.244265,0.716075
3,smut,1,4,-0.346142,-0.443261,-1.486278,-0.374852,-0.507371,0.219723,1.382270,...,1.734046,-0.834634,-0.167471,-0.726386,2.582892,-0.216003,-0.687492,-0.381337,-0.510485,-0.354114


In [21]:
intermediate_layer_model = tf.keras.Model(inputs=model.input,
                                         outputs=model.get_layer('multi_head_qkv__block_attention').output)
intermediate_output = intermediate_layer_model.predict(data)
attention = intermediate_output[0]
attention = K.expand_dims(attention,axis=1)
print(attention.shape)
pd.DataFrame(np.array(tf.reshape(attention,(32,2608)))).describe()

(2608, 1, 32)


,0,1,2,3,4,5,6,7,8,9,...,2598,2599,2600,2601,2602,2603,2604,2605,2606,2607
count,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,...,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000
mean,0.025982,-0.039928,-0.040407,0.034194,-0.028745,0.087532,-0.052552,0.028997,-0.003427,0.022198,...,-0.052552,0.028997,-0.003427,0.022198,-0.040022,0.029404,0.095634,0.049117,-0.036483,0.100009
std,0.002597,0.042523,0.015336,0.029727,0.001629,0.016146,0.024049,0.012860,0.035353,0.003094,...,0.024049,0.012860,0.035353,0.003094,0.013306,0.053255,0.023715,0.039082,0.016013,0.041146
min,0.023426,-0.081781,-0.055501,0.004935,-0.030349,0.071640,-0.076222,0.016339,-0.038224,0.019152,...,-0.076222,0.016339,-0.038224,0.019152,-0.053118,-0.023012,0.072292,0.010651,-0.052244,0.059512
25%,0.023426,-0.081781,-0.055501,0.004935,-0.030349,0.071640,-0.076222,0.016339,-0.038224,0.019152,...,-0.076222,0.016339,-0.038224,0.019152,-0.053118,-0.023012,0.072292,0.010651,-0.052244,0.059512
50%,0.025982,-0.039928,-0.040407,0.034194,-0.028745,0.087532,-0.052552,0.028997,-0.003427,0.022198,...,-0.052552,0.028997,-0.003427,0.022198,-0.040022,0.029404,0.095634,0.049117,-0.036483,0.100009
75%,0.028538,0.001925,-0.025312,0.063452,-0.027142,0.103424,-0.028882,0.041654,0.031370,0.025243,...,-0.028882,0.041654,0.031370,0.025243,-0.026926,0.081821,0.118975,0.087583,-0.020723,0.140507
max,0.028538,0.001925,-0.025312,0.063452,-0.027142,0.103424,-0.028882,0.041654,0.031370,0.025243,...,-0.028882,0.041654,0.031370,0.025243,-0.026926,0.081821,0.118975,0.087583,-0.020723,0.140507
